Assignment 3

In [ ]:
import requests

In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
website_url = requests.get(url).text

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup(website_url,'lxml')
print(soup.prettify())

In [ ]:
my_table = soup.find('table',{'class':'wikitable sortable'})
print(my_table)

In [ ]:
links = my_table.findAll('td')
links[0]

<td>M1A</td>

In [ ]:
f'number of rows in our table is: {len(links)/3}'

'number of rows in our table is: 287.0'

In [ ]:
Postcodes = [] 
Boroughs = [] 
neighborhoods = []

i = 0

for word in links:
  if i%3 == 0:
    Postcodes.append(word.get_text().strip())
    
  elif i%3 == 1:
    Boroughs.append(word.get_text().strip())
  
  elif i%3 == 2:
    neighborhoods.append(word.get_text().strip())
  i+=1
Boroughs
    

In [ ]:
len(neighborhoods) == len(Boroughs) == len(Postcodes)

True

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame()
df['Postalcodes'] = Postcodes
df['Borough'] = Boroughs
df['neighborhood'] = neighborhoods
df[df['Postalcodes']== 'M1R']

,Postalcodes,Borough,neighborhood
163,M1R,Scarborough,Maryvale
164,M1R,Scarborough,Wexford


In [ ]:
indexes = []
for index, row in df.iterrows():
  if row['Borough'] == 'Not assigned':
    indexes.append(index)
    continue
  elif row['neighborhood'] == 'Not assigned':
    row['neighborhood'] == row['Borough']
df.head()

,Postalcodes,Borough,neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [ ]:
df.drop(indexes, inplace=True)
df.head()

,Postalcodes,Borough,neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [ ]:
df1 = df.groupby('Postalcodes')['neighborhood'].apply(', '.join).reset_index()
df1.head()

,Postalcodes,neighborhood
0,M1B,"Rouge, Malvern"
1,M1C,"Highland Creek, Rouge Hill, Port Union"
2,M1E,"Guildwood, Morningside, West Hill"
3,M1G,Woburn
4,M1H,Cedarbrae


In [ ]:
df = pd.merge( df[['Postalcodes','Borough']], df1[['Postalcodes','neighborhood']], on='Postalcodes')
df.head()

,Postalcodes,Borough,neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M6A,North York,"Lawrence Heights, Lawrence Manor"


In [ ]:
df.shape

(210, 3)

Latitude and Longitude for our Postal codes using this csv file:
http://cocl.us/Geospatial_data

In [ ]:
!wget -O geospatial_data.csv 'http://cocl.us/Geospatial_data'

--2019-12-16 15:05:36--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 161.202.50.39
Connecting to cocl.us (cocl.us)|161.202.50.39|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2019-12-16 15:05:36--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|161.202.50.39|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-12-16 15:05:38--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 103.116.4.197
Connecting to ibm.box.com (ibm.box.com)|103.116.4.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-12-16 15:05:39--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5m

In [ ]:
geospatial_data = pd.read_csv('geospatial_data.csv')
geospatial_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [ ]:
geospatial_data = geospatial_data.rename(columns={"Postal Code": "Postalcodes"}, errors="raise")
geospatial_data.head()

,Postalcodes,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [ ]:
df = df.merge(geospatial_data, on='Postalcodes')
                                                        
df.head()

,Postalcodes,Borough,neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763


In [ ]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 11 boroughs and 210 neighborhoods.


In [ ]:
import folium
from geopy.geocoders import Nominatim

In [ ]:
address = 'Toronto'

geolocator = Nominatim(user_agent="t_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [ ]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto